In [3]:
import mobster as mb

In [4]:
data = mb.load_example_data("./")

In [5]:
fitted = mb.fit_mobster(data, 1,1, max_it = 500, lr = 0.01)

Running MOBSTER on 3 karyotypes with 1 subclones.
Fitting a model with tail
Computing cluster assignements.
Computing information criteria.
Done!


ELBO: -5575.775779376  : 100%|██████████| 500/500 [00:14<00:00, 33.55it/s]


In [8]:
mb.plot_results(data, fitted, output= "cane.png")